In [32]:
from typing import List,TypedDict, Any, Optional, Dict
from langchain_ollama import ChatOllama
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import HumanMessage

In [33]:
class LeakState(TypedDict):
    leak: Dict[str, Any]
    leak_category: Optional[str]
    is_legit: Optional[bool]
    non_legit_reason: Optional[str]
    post_draft: Optional[str]

    messages: List[Dict[str, Any]]

In [34]:
from langchain_community.tools.ddg_search.tool import DuckDuckGoSearchRun

# Create a searcher instance (no API key needed)
duck_search = DuckDuckGoSearchRun()

from langchain_core.tools import Tool

web_search_tool = Tool(
    name="web_search",
    description="Search the web via DuckDuckGo for leak verification",
    func=duck_search.run,
)

In [35]:
# LLM Initialize
model = ChatOllama(
    model = 'llama3.1:latest',
    # api_base = '127.0.0.1:14343',
)

def read_leak(state: LeakState):
    """Reads through the sent leak"""
    leak = state['leak']
    print(f"The leak processor is reading the leak from {leak['sender']} with subject {leak['subject']}")

    return {}

def classify_leak(state: LeakState):
    """Determines if the leak is legit or not"""
    leak = state['leak']
    prompt = f"""
    As a leak assistant for HoyoVerse, you have to search if the leak presented is legitimate or fake
    Leak:
    From: {leak['sender']}
    Subject: {leak['subject']}
    Body: {leak['body']}
    First, determine (search) if this leak is legitimate (real) or not. If it is not legit, explain why.
    If it is legitimate, categorize it by the game (Genshin Impact, Honkai: Star Rail, Zenless Zone Zero).
    """

    messages = [HumanMessage(content=prompt)]
    response = model.invoke(messages)

    response_text = response.content.lower()
    is_legit = "legit" in response_text and "not legit" not in response_text

    non_legit_reason = None
    if not is_legit and "reason:" in response_text:
        non_legit_reason = response_text.split("reason:")[1].strip()

    leak_category = None
    if is_legit:
        categories = ['Genshin Impact', 'Honkai: Star Rail', 'Zenless Zone Zero']
        for category in categories:
            if category in response_text:
                leak_category = category
                break

    new_messages = state.get('messages', []) + [
        {'role': 'user', 'content': prompt},
        {'role': 'assistant', 'content': response.content}
    ]

    return {
        "is_legit": is_legit,
        "non_legit_reason": non_legit_reason,
        "leak_category": leak_category,
        "messages": new_messages
    }

def handle_non_legit(state: LeakState):
    """Discards the non-legit leaks with a note"""
    print(f"I have marked this leak as non-legit. Reason: {state['non_legit_reason']}")
    print("This leak has been discarded and moved to spam folder")
    
    return {}

def draft_post(state: LeakState):
    """Drafts a post to be posted on Telegram channel"""
    leak = state['leak']
    category = state['leak_category'] or "general"

    prompt = f"""
    Draft a post for my Telegram leak channel according to this leak:

    Leak:
    From: {leak['sender']}
    Subject: {leak['subject']}
    Body: {leak['body']}

    This leak has been categorized as: {category}

    Draft a brief, interesting summary of the leak to be posted on the Telegram channel.
    """

    messages = [HumanMessage(content=prompt)]
    response = model.invoke(messages)

    new_messages = state.get("messages", []) + [
        {'role':'user', 'content':prompt},
        {'role':'assistant', 'content': response.content}
    ]

    return {
        'post_draft': response.content,
        'messages': new_messages
    }

def notify_mod(state: LeakState):
    """Notifies the channel moderator about the new leak and drafts the post."""
    leak = state['leak']

    print("\n" + "="*50)
    print(f"We have received yet another leak from {leak['sender']}.")
    print(f"Subject: {leak['subject']}")
    print(f"Category: {state['leak_category']}")
    print("\nI've drafted a post for your review:")
    print("-"*50)
    print(state['post_draft'])
    print("="*50 + "\n")

    return {}

In [36]:
def route_leak(state: LeakState) -> str:
    """Determine the next step based on the legit classification"""
    if state["is_legit"]:
        return "legitimate"
    else:
        return "non-legit"

In [39]:
leak_graph = StateGraph(LeakState)

leak_graph.add_node("read_leak", read_leak)
leak_graph.add_node("classify_leak", classify_leak)
leak_graph.add_node("handle_non_legit", handle_non_legit)
leak_graph.add_node("draft_post", draft_post)
leak_graph.add_node("notify_mod", notify_mod)

leak_graph.add_edge(START, "read_leak")
leak_graph.add_edge("read_leak", "classify_leak")
leak_graph.add_conditional_edges(
    "classify_leak",
    route_leak,
    {
        "legitimate": "draft_post",
        "non-legit": "handle_non_legit"
    }
)

leak_graph.add_edge("draft_post", "notify_mod")
leak_graph.add_edge("notify_mod", END)
leak_graph.add_edge("handle_non_legit", END)

compiled_graph = leak_graph.compile()

In [40]:
legit_genshin_leak = {
    "sender": "Honey Impact",
    "subject": "About new character",
    "body": "Skirk is gonna be one of the characters featured on the first side of 5.7 character banners."
}

non_legit_genshin_leak = {
    "sender": "Dodogama",
    "subject": "Il Capitano Release",
    "body": "Il Capitano is going to be released in the second half of patch 5.7."
}

print("\nProcessing the legit leak...")
legitimate_result = compiled_graph.invoke({
    "leak": legit_genshin_leak,
    "is_legit": None,
    "leak_category": None,
    "non_legit_reason": None,
    "post_draft": None,
    "messages": []
})

print("\nProcessing the non-legit leak...")
non_legit_result = compiled_graph.invoke({
    "leak": non_legit_genshin_leak,
    "leak": legit_genshin_leak,
    "is_legit": None,
    "leak_category": None,
    "non_legit_reason": None,
    "post_draft": None,
    "messages": []
})


Processing the legit leak...
The leak processor is reading the leak from Honey Impact with subject About new character
I have marked this leak as non-legit. Reason: None
This leak has been discarded and moved to spam folder

Processing the non-legit leak...
The leak processor is reading the leak from Honey Impact with subject About new character

We have received yet another leak from Honey Impact.
Subject: About new character
Category: None

I've drafted a post for your review:
--------------------------------------------------
Here's a draft post for your Telegram leak channel:

**NEW LEAK ALERT!**

Honey Impact just dropped a bombshell! According to their latest scoop, **Skirk** will be one of the characters featured on the first side of 5.7 character banners!

Stay tuned for more updates and get ready for some exciting new character reveals!

**Source:** Honey Impact
**Category:** General

Share with your fellow gamers and let's discuss!

